# Canopy Height Model

Canopy height models (CHM's) are created by finding the difference between a digital surface model (DSM) and a digital terrain model (DTM). CHM's present the height of trees in the area of interest. They can be used to assist in the identification of tree species, suhc as Kauri. Mature Kauri can be 30-50m tall, typically larger than surrounding species. 

This notebook will explain how to create a CHM of an area from LiDAR data. There will be four key steps in achieving this: 

+ Import data
+ Merge raster tiles
+ Calculate CHM
+ Present results in a visual format


### Import Data
LiDAR data is made up of tiles. When downloading data for an area it is likely to be made up of mulitple tiles. Loading each tile individually would be time consuming so the following code chunk will explain how to import a single tile, and then how to import mulitple tiles efficiently. 


In [ ]:
# first run this cell to import all the required libraries
# You need to run this cell to get things setup
%matplotlib inline
import matplotlib
import geopandas
import matplotlib.pyplot as plt        
import rasterio as rio  
from rasterio.merge import merge
from rasterio.plot import show
import subprocess, glob
from osgeo import gdal
import os                                    
import sys                             
import numpy as np                      
                               



#### Import One Tile

In [ ]:
#import one DTM tile
dtm=rio.open('chm_data\DEM_BB31_1002_2013.tif')
show(dtm)
# this is the only line changed

#### Import More Than One Tile

In [ ]:
#import multiple DTM tiles
DTMsearch_criteria = "DEM*.tif" #find all .tif files that start DEM
DTMpath='chm_data' #folder where the data is
DTM_pathALL= os.path.join(DTMpath, DTMsearch_criteria) #join folder and possible .tif files together
dtm_all= glob.glob(DTM_pathALL) #open all files in the data folder than match the search criteria

#dtm_all #check to see if all .tif files have been opened

The above code imports the .tif files into a list but they have not been opened yet.                                                       

#### Opening Files 

In [ ]:
dtm_mosaic = [] #create empty list where the mosaic will do

for fp in dtm_all: #for each file in the list of dtm .tif files
    dtm = rio.open(fp) #open the file
    dtm_mosaic.append(dtm) #and append it to the list

#dtm_mosaic #demonstrating that the files have now been opened

### Merge Files

In [ ]:
# mosaicing the files together
dtm_mosaic, out_trans = merge(dtm_mosaic) #the files in the list are merged together

show(dtm_mosaic, cmap='terrain') #visualising dtm

#### DSM
The same process will be repeated for the DSM

In [ ]:
#path to folder
#change to relevant folder for your computer
DSM_path = 'chm_data'

#takes all tif files that start with DSM
DSM_search_criteria = "DSM*.tif"

#combining folder path and file name
z = os.path.join(DSM_path, DSM_search_criteria)
#loading all files
dsm_all= glob.glob(z)

#combining files into a mosaic
#start with empty list
dsm_files_to_mosaic = []
#for each file in the list of files, open it and combine it with the others 
for fp1 in dsm_all:
    dsm = rio.open(fp1)
    dsm_files_to_mosaic.append(dsm)

# Merge function returns a single mosaic array and the transformation info
dsm_mosaic, out_trans = merge(dsm_files_to_mosaic)
#display mosaic
show(dsm_mosaic, cmap='terrain')

Now that mosaics of the DTM and DSM have been created from the LiDAR data the CHM will be created using the following equation:

CHM = DSM-DTM

In [ ]:
# Calculate canopy height model
chm = dsm_mosaic - dtm_mosaic

### Visualise the CHM

In [ ]:
fig, ax = plt.subplots(figsize = (10,10))
show(chm, cmap='viridis', 
     title="Canopy Height Model for the Waitakere Ranges", 
     ax=ax)
ax.set_axis_off()

#### Histogram of Canopy Height

In [ ]:
from rasterio.plot import show_hist
fig, ax = plt.subplots(figsize=(10,10))
show_hist(chm, ax=ax, bins=100, lw=0.0, stacked=False, alpha=0.3, title="Histogram CHM")
plt.show()

A histogram is another method of visualising the CHM allowing for the frequency of different heights to be investigated. Here the DN is the height of the canopy in meters. 

**Location of tall trees**

Kauri trees commonly grow greater than 20m tall. In the histogram it is possible to see that there is only a small proportionn of cells with a height greater than 20m. Determining the location of where these trees are may assist with identifying Kauri trees

In [ ]:
chm_20=chm>20 #create new variable for cells that have a height value greater than 20m
show(chm_20) #plot this variable